In [4]:
import requests as rq
import pymongo as pm
import ujson as js
import pandas as pd
from datetime import datetime as dt

from ApiHandler import ApiHandler

In [62]:
class ApiWalletJournal(ApiHandler):
    settings = {
        **ApiHandler.settings,
    }
    url = {
        **ApiHandler.url,
        'data': 'https://esi.evetech.net/latest/corporations/{corp_id}/wallets/{division}/journal/'
    }
    script_vals = {
        **ApiHandler.script_vals,
        'table': 'Corp_WalletJournal',
    }
    name = 'ESI Wallet Journal API'
    params = {}
    auth = True
    delete = False
    
    def get_raw_data(self):
        if self.verbose: self._verbose('get_raw_data', 'Getting raw data...')
        
        if self.auth:
            self.headers['Authorization'] = 'Bearer {token}'.format(**{
                'token': self.auth_token
            })
        agg_data = []
        for division in range(1,8):
            self.url_params['division'] = division
            data_conn = self.url_get(
                self.url['data'].format(**self.url_params),
                self.headers,
                self.params
            )

            self.timestamp = dt.strptime(
                data_conn.headers['Last-Modified'],
                '%a, %d %b %Y %H:%M:%S %Z'
            ).strftime('%Y-%m-%d %H:%M:%S')

            page_count = int(data_conn.headers.get('X-Pages', '1'))
            if page_count != 1:
                if self.verbose: self._verbose('get_raw_data', 'Multi-page raw data...')
                data_conn = [data_conn.content]
                for page in range(2, page_count+1):
                    data_conn.append(self.url_get(
                        self.url['data'].format(**self.url_params),
                        self.headers,
                        {**self.params, 'page': page}
                    ).content)
            else:
                data_conn = data_conn.content
                
            agg_data.append(data_conn)

        if self.verbose: self._verbose('get_raw_data', 'Raw data acquired.')
        
        return agg_data
    
    def build_data(self, raw_data):
        data_frame = []
        for data_idx in range(len(raw_data)):
            data = raw_data[data_idx]
            if type(data) == list:
                data = pd.concat([pd.DataFrame(js.loads(data_item)) for data_item in data], axis=0, sort=False)
            else:
                data = pd.DataFrame(js.loads(data))
            data['division'] = data_idx
            data['corporation_id'] = self.url_params['corp_id']
            data_frame.append(data)
        data_frame = pd.concat(data_frame, sort = True)
        data_frame['date'] = data_frame['date'].apply(lambda x: dt.strptime(
            x, '%Y-%m-%dT%H:%M:%SZ'
        ).strftime('%Y-%m-%d %H:%M:%S'))
        data_frame.rename(columns={
            'id':'journal_id',
            'date':'journal_date'
        }, inplace=True)
        return data_frame

In [64]:
api = ApiWalletJournal()
api.run_process()